In [ ]:
# 1. Clean environment
!pip uninstall -y torch torchvision torchaudio torchtext torchdata

# 2.  mamba-ssm cp312 wheel  torch 2.4.0
!pip install torch==2.4.0+cu118 torchvision==0.19.0+cu118 torchaudio==2.4.0+cu118 \
  --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# 3.  cp312 wheel（Avoid compilation; install prebuilt wheel）
!pip install \
  https://github.com/state-spaces/mamba/releases/download/v2.2.6.post3/mamba_ssm-2.2.6.post3+cu11torch2.4cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

import torch, mamba_ssm
print(torch.__version__)
print(mamba_ssm.__version__)

In [ ]:
import os
import math
import random
import numpy as np
import pandas as pd
from typing import List, Dict

import matplotlib.pyplot as plt
import torch


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

CSV_PATH = "PATH/processed_data.csv"  #  Colab 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

df = pd.read_csv(CSV_PATH)

df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")

print("ts dtype:", df["ts"].dtype)
print(df.head(3))

print("Columns:", df.columns.tolist())
print(df.head(3))

#  vehicle_id 
df["car_no"] = df["vehicle_id"].str[-3:].astype(int)

#  Race 1 Sonoma 
top_cars = [46, 13, 55, 2, 16, 78, 98, 88, 71, 31]
df_top = df[df["car_no"].isin(top_cars)].copy()

#  flying （90–120s ，，）
#  all_merged_with_dist_progress  flying-only + strict
#  is_true_flying/in_90_120 ，：
# df_top = df_top[df_top["is_true_flying"] == True]

# 
lap_times = (
    df_top
    .groupby(["vehicle_id", "car_no", "lap"])["ts"]
    .agg(lambda x: (x.iloc[-1] - x.iloc[0]).total_seconds())
    .reset_index(name="lap_time_real")
)

# Pick the fastest lap for each vehicle 3 
best3 = (
    lap_times
    .sort_values(["vehicle_id", "lap_time_real"])
    .groupby("vehicle_id")
    .head(3)
    .reset_index(drop=True)
)
print("3：")
print(best3)

#  is_best_lap 
keys = best3[["vehicle_id", "lap"]].copy()
keys["is_best_lap"] = True

df_best = df_top.merge(keys, on=["vehicle_id", "lap"], how="inner")
df_best = df_best.sort_values(["vehicle_id", "lap", "ts"]).reset_index(drop=True)

print("：")
print(df_best.head(5))


In [ ]:
# Select a reference car: 46 (GR86-033-046)
ref_vid = best3[best3["car_no"] == 46]["vehicle_id"].iloc[0]
ref_lap = best3[best3["vehicle_id"] == ref_vid].sort_values("lap_time_real")["lap"].iloc[0]

print(":", ref_vid, "lap", ref_lap)

ref = df_best[(df_best["vehicle_id"] == ref_vid) & (df_best["lap"] == ref_lap)].copy()
ref = ref.sort_values("ts").reset_index(drop=True)

# G
ref["steer_smooth"] = ref["Steering_Angle"].rolling(7, center=True, min_periods=1).mean()
ref["accy_smooth"] = ref["accy_can"].rolling(7, center=True, min_periods=1).mean()

#  corner : 5  0.2 
steer_thresh = 5.0
accy_thresh = 0.2

corner_mask = (ref["steer_smooth"].abs() > steer_thresh) | (ref["accy_smooth"].abs() > accy_thresh)

corners = []
start = None

for i, flag in enumerate(corner_mask):
    if flag and start is None:
        start = i
    # ：，
    if (not flag or i == len(corner_mask) - 1) and start is not None:
        end = i
        dt = (ref.loc[end, "ts"] - ref.loc[start, "ts"]).total_seconds()
        if dt >= 0.4:  # 
            corners.append((start, end, dt))
        start = None

print(" corner :", len(corners))

corner_rows = []
for idx, (s, e, dt) in enumerate(corners, start=1):
    seg = ref.loc[s:e]
    corner_rows.append({
        "corner_index": idx,
        "idx_start": s,
        "idx_end": e,
        "dt_corner": dt,
        "progress_start": seg["progress_dist"].iloc[0],
        "progress_end": seg["progress_dist"].iloc[-1],
        "steer_mean": seg["steer_smooth"].mean(),
        "steer_sign": "right" if seg["steer_smooth"].mean() > 0 else "left",
        "speed_min": seg["speed"].min(),
        "speed_max": seg["speed"].max(),
    })

corner_map = pd.DataFrame(corner_rows)
print(" progress_dist ：")
print(corner_map[["corner_index","progress_start","progress_end","steer_sign","dt_corner","speed_min"]])


In [ ]:
# Assign corner_index for all best laps corner_index
def assign_corner_index(row, corner_map):
    p = row["progress_dist"]
    #  progress_start <= p <= progress_end 
    matches = corner_map[(p >= corner_map["progress_start"]) & (p <= corner_map["progress_end"])]
    if len(matches) == 0:
        return 0  # Not inside any corner，
    # ， progress_start 
    return int(matches.sort_values("progress_start").iloc[-1]["corner_index"])

df_best["corner_index"] = df_best.apply(assign_corner_index, axis=1, corner_map=corner_map)

print(df_best[["vehicle_id","lap","ts","progress_dist","Steering_Angle","corner_index"]].head(20))


In [ ]:
# ：
# df       = processed_data.csv 、 progress_dist、ts 
# df["car_no"] = ...
# df_top   = df[df["car_no"].isin(top_cars)].copy()
# corner_map （progress_start / progress_end ）

# 1)  df_top  corner_index（ best laps ）
def assign_corner(row, corner_map):
    p = row["progress_dist"]
    matches = corner_map[
        (p >= corner_map["progress_start"]) &
        (p <= corner_map["progress_end"])
    ]
    if len(matches) == 0:
        return 0  # 
    return int(matches.sort_values("progress_start").iloc[-1]["corner_index"])

df_top = df_top.sort_values(["vehicle_id","lap","ts"]).reset_index(drop=True)
df_top["corner_index"] = df_top.apply(assign_corner, axis=1, corner_map=corner_map)

# 2)  Steering （ window=5）
df_top["Steering_smooth"] = (
    df_top
    .groupby(["vehicle_id","lap"])["Steering_Angle"]
    .transform(lambda s: s.rolling(window=5, center=True, min_periods=1).mean())
)

# 
df_top["ath_smooth"] = (
    df_top
    .groupby(["vehicle_id", "lap"])["ath"]
    .transform(lambda s: s.rolling(window=5, center=True, min_periods=1).mean())
)

# 
df_top["pbrake_f_smooth"] = (
    df_top
    .groupby(["vehicle_id", "lap"])["pbrake_f"]
    .transform(lambda s: s.rolling(window=5, center=True, min_periods=1).mean())
)


print(df_top[[
    "vehicle_id","lap","ts",
    "Steering_Angle","Steering_smooth",
    "ath","ath_smooth",
    "pbrake_f","pbrake_f_smooth"
    ]].head(10))


In [ ]:
# ====== STEP3:  Steering （label smoothing） ======
# ：
# - 、 rolling，Avoid contamination across laps
# - window=5 （，）

df_best = df_best.sort_values(["vehicle_id", "lap", "ts"]).reset_index(drop=True)

df_best["Steering_smooth"] = (
    df_best
    .groupby(["vehicle_id", "lap"])["Steering_Angle"]
    .transform(lambda s: s.rolling(window=5, center=True, min_periods=1).mean())
)

# 
df_best["ath_smooth"] = (
    df_best
    .groupby(["vehicle_id", "lap"])["ath"]
    .transform(lambda s: s.rolling(window=5, center=True, min_periods=1).mean())
)

# 
df_best["pbrake_f_smooth"] = (
    df_best
    .groupby(["vehicle_id", "lap"])["pbrake_f"]
    .transform(lambda s: s.rolling(window=5, center=True, min_periods=1).mean())
)

print(df_best[[
    "vehicle_id","lap","ts",
    "Steering_Angle","Steering_smooth",
    "ath","ath_smooth",
    "pbrake_f","pbrake_f_smooth"
]].head(10))



In [ ]:

df_best = df_best.sort_values(["vehicle_id", "lap", "ts"]).reset_index(drop=True)

# ： progress_dist  progress， corner_index
OBS_FEATURES = [
    "progress_dist",       # 
    "corner_index",        # （0 = ，1,2,... = ）
    "speed",
    "accx_can",
    "accy_can",
    "nmot",
    "gear",
]

# 
ACT_FEATURES = ["Steering_smooth", "ath_smooth", "pbrake_f_smooth"]



print(":", OBS_FEATURES)
print(":", ACT_FEATURES)

sequences: List[Dict] = []

for (vid, lap), g in df_best.groupby(["vehicle_id", "lap"]):
    g = g.reset_index(drop=True)
    obs = g[OBS_FEATURES].values.astype(np.float32)
    act = g[ACT_FEATURES].values.astype(np.float32)
    if len(g) < 5:
        continue
    sequences.append({
        "vehicle_id": vid,
        "lap": int(lap),
        "obs": obs,
        "act": act,
    })

print(":", len(sequences))

import random
random.seed(42)
indices = list(range(len(sequences)))
random.shuffle(indices)
split = int(len(indices) * 0.8)
train_idx = indices[:split]
val_idx = indices[split:]

train_seqs = [sequences[i] for i in train_idx]
val_seqs   = [sequences[i] for i in val_idx]

print(f": {len(train_seqs)}, : {len(val_seqs)}")


obs_train_concat = np.concatenate([s["obs"] for s in train_seqs], axis=0)
act_train_concat = np.concatenate([s["act"] for s in train_seqs], axis=0)

obs_mean = obs_train_concat.mean(axis=0, keepdims=True)
obs_std  = obs_train_concat.std(axis=0, keepdims=True) + 1e-6

act_mean = act_train_concat.mean(axis=0, keepdims=True)
act_std  = act_train_concat.std(axis=0, keepdims=True) + 1e-6

def normalize_obs(x: np.ndarray) -> np.ndarray:
    return (x - obs_mean) / obs_std

def normalize_act(y: np.ndarray) -> np.ndarray:
    return (y - act_mean) / act_std

for s in train_seqs:
    s["obs_norm"] = normalize_obs(s["obs"])
    s["act_norm"] = normalize_act(s["act"])

for s in val_seqs:
    s["obs_norm"] = normalize_obs(s["obs"])
    s["act_norm"] = normalize_act(s["act"])

# ============== Dataset & DataLoader（） ==============

class DrivingSequenceDataset(Dataset):
    def __init__(self, seqs):
        self.seqs = seqs

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        s = self.seqs[idx]
        obs = s["obs_norm"]
        act = s["act_norm"]
        length = obs.shape[0]
        return {
            "obs": torch.from_numpy(obs),
            "act": torch.from_numpy(act),
            "length": length,
        }

def collate_fn(batch):
    max_len = max(item["length"] for item in batch)
    batch_size = len(batch)
    D_obs = batch[0]["obs"].shape[1]
    D_act = batch[0]["act"].shape[1]

    obs_batch = torch.zeros(batch_size, max_len, D_obs, dtype=torch.float32)
    act_batch = torch.zeros(batch_size, max_len, D_act, dtype=torch.float32)
    mask = torch.zeros(batch_size, max_len, dtype=torch.bool)

    for i, item in enumerate(batch):
        L = item["length"]
        obs_batch[i, :L] = item["obs"]
        act_batch[i, :L] = item["act"]
        mask[i, :L] = True

    return {
        "obs": obs_batch.to(device),
        "act": act_batch.to(device),
        "mask": mask.to(device),
        "lengths": torch.tensor([item["length"] for item in batch], dtype=torch.long).to(device),
    }

train_dataset = DrivingSequenceDataset(train_seqs)
val_dataset   = DrivingSequenceDataset(val_seqs)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)


In [ ]:
# best3 best3 already contains the top 3 fastest laps of each car： vehicle_id, car_no, lap, lap_time_real

# 1)  best laps 
best_keys = set((row["vehicle_id"], int(row["lap"])) for _, row in best3.iterrows())

# 2)  df_top “ best3” = 
slow_candidates = []
for (vid, lap), g in df_top.groupby(["vehicle_id","lap"]):
    key = (vid, int(lap))
    if key in best_keys:
        continue  # 
    if len(g) < 5:
        continue  # 
    slow_candidates.append((vid, int(lap), g["ts"].iloc[0]))

print(":", len(slow_candidates))
print(":", slow_candidates[:10])

# OBS_FEATURES  ACT_FEATURES ：
OBS_FEATURES = [
    "progress_dist",
    "corner_index",
    "speed",
    "accx_can",
    "accy_can",
    "nmot",
    "gear",
]

ACT_FEATURES = ["Steering_smooth", "ath_smooth", "pbrake_f_smooth"]
# ACT_FEATURES = ["Steering_Angle", "ath", "pbrake_f"]
slow_seqs = []

for (vid, lap, _) in slow_candidates:
    g = df_top[(df_top["vehicle_id"] == vid) & (df_top["lap"] == lap)].copy()
    g = g.sort_values("ts").reset_index(drop=True)
    obs = g[OBS_FEATURES].values.astype(np.float32)
    act = g[ACT_FEATURES].values.astype(np.float32)
    if len(g) < 5:
        continue

    obs_norm = (obs - obs_mean) / obs_std
    act_norm = (act - act_mean) / act_std

    slow_seqs.append({
        "vehicle_id": vid,
        "lap": int(lap),
        "obs": obs,
        "act": act,
        "obs_norm": obs_norm,
        "act_norm": act_norm,
        "corner_index": g["corner_index"].values.astype(np.int32),
        "progress_dist": g["progress_dist"].values.astype(np.float32),  # 
        # ，
        "ath": g["ath"].values.astype(np.float32),
        "pbrake_f": g["pbrake_f"].values.astype(np.float32),
    })


print(" slow_seqs :", len(slow_seqs))
if slow_seqs:
    print(":", slow_seqs[0]["vehicle_id"], "lap", slow_seqs[0]["lap"], "len", slow_seqs[0]["obs"].shape[0])



In [ ]:

# ============================
# 6. Model Structure
# ============================

input_dim = len(OBS_FEATURES)
output_dim = len(ACT_FEATURES)
hidden_dim = 64
num_layers = 2

from mamba_ssm import Mamba

class DrivingMamba(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, d_state=16, d_conv=4, expand=2):
        super().__init__()
        self.mamba = Mamba(
            d_model=hidden_dim,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand,
        )
        self.in_proj = nn.Linear(input_dim, hidden_dim)
        self.out_proj = nn.Linear(hidden_dim, output_dim)

    def forward(self, obs, lengths=None):
        # obs: (B, T, D_in)
        x = self.in_proj(obs)        # (B, T, hidden_dim)
        # Mamba  (B, T, hidden_dim)
        y = self.mamba(x)            # (B, T, hidden_dim)
        out = self.out_proj(y)       # (B, T, D_out)
        return out


model = DrivingMamba(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.MSELoss(reduction="none")  #  mask 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print(model)


In [ ]:
# ============================
# Tools
# ============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def masked_mse_loss(pred, target, mask):
    diff2 = (pred - target) ** 2  # (B, T, D)
    diff2 = diff2.mean(dim=-1)    # (B, T)
    diff2 = diff2 * mask.float()
    loss = diff2.sum() / mask.float().sum().clamp(min=1.0)
    return loss

def run_one_epoch(dataloader, training=True):

    model.train() if training else model.eval()

    total_loss = 0.0
    total_count = 0

    for batch in dataloader:
        obs = batch["obs"].to(device)     # <-- 
        act = batch["act"].to(device)     # <-- 
        mask = batch["mask"].to(device)   # <-- 
        lengths = batch["lengths"].to(device)  # 

        if training:
            optimizer.zero_grad()

        pred = model(obs, lengths)  # (B, T, D_act)

        loss = masked_mse_loss(pred, act, mask)

        if training:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        #  mask.float()  .cpu()  python float
        count = mask.float().sum().item()

        total_loss += loss.item() * count
        total_count += count

    if total_count == 0:
        return float("inf")

    return total_loss / total_count


In [ ]:
EPOCHS = 240 

for epoch in range(1, EPOCHS+1):
    train_loss = run_one_epoch(train_loader, training=True)
    val_loss = run_one_epoch(val_loader, training=False)

    print(f"Epoch {epoch:02d} | train_loss = {train_loss:.6f} | val_loss = {val_loss:.6f}")


In [ ]:
def analyze_one_lap_diff(seq, model, act_mean, act_std, title=""):
    """
    seq: slow_seqs 
         : obs_norm, act_norm, corner_index, vehicle_id, lap
         act_norm: (T, 3)   [Steering_smooth, ath_smooth, pbrake_f_smooth]
    """
    model.eval()

    obs_np = seq["obs_norm"]           # (T, D_obs)
    true_act_norm = seq["act_norm"]    # (T, 3)
    corner_idx = seq["corner_index"]   # (T,)

    obs = torch.from_numpy(obs_np).unsqueeze(0).to(device)
    lengths = torch.tensor([obs_np.shape[0]], dtype=torch.long).to(device)

    with torch.no_grad():
        pred_norm = model(obs, lengths)    # (1, T, 3)

    pred_norm_np = pred_norm.squeeze(0).cpu().numpy()   # (T, 3)
    true_norm_np = true_act_norm                         # (T, 3)

    # 
    true_phys = true_norm_np * act_std + act_mean        # (T, 3)
    pred_phys = pred_norm_np * act_std + act_mean        # (T, 3)

    steer_true = true_phys[:, 0]
    ath_true   = true_phys[:, 1]
    brk_true   = true_phys[:, 2]

    steer_pred = pred_phys[:, 0]
    ath_pred   = pred_phys[:, 1]
    brk_pred   = pred_phys[:, 2]

    # Difference of the three actions
    diff_steer = steer_pred - steer_true
    diff_ath   = ath_pred   - ath_true
    diff_brk   = brk_pred   - brk_true

    T = steer_true.shape[0]
    x = np.arange(T)

    vid = seq.get("vehicle_id", "N/A")
    lap = seq.get("lap", -1)

    # =================  =================
    plt.figure(figsize=(14, 12))

    # 1) Steering  vs 
    plt.subplot(6,1,1)
    plt.plot(x, steer_true, label="True Steering", linewidth=1.2)
    plt.plot(x, steer_pred, label="Pred Steering", linestyle="--", linewidth=1.2)
    plt.ylabel("Steer (deg)")
    plt.title(f"{title}  {vid} lap {lap}")
    plt.legend()
    plt.grid(True)

    # 2) Steering diff
    plt.subplot(6,1,2)
    plt.plot(x, diff_steer, label="Δ Steering (pred-true)", linewidth=1.2)
    plt.axhline(0, color="black", linewidth=0.8)
    plt.ylabel("Δ Steer")
    plt.legend()
    plt.grid(True)

    # 3) Throttle  vs 
    plt.subplot(6,1,3)
    plt.plot(x, ath_true, label="True Throttle", linewidth=1.2)
    plt.plot(x, ath_pred, label="Pred Throttle", linestyle="--", linewidth=1.0)
    plt.ylabel("Throttle")
    plt.legend()
    plt.grid(True)

    # 4) Throttle diff
    plt.subplot(6,1,4)
    plt.plot(x, diff_ath, label="Δ Throttle (pred-true)", linewidth=1.2)
    plt.axhline(0, color="black", linewidth=0.8)
    plt.ylabel("Δ Throttle")
    plt.legend()
    plt.grid(True)

    # 5) Brake  vs 
    plt.subplot(6,1,5)
    plt.plot(x, brk_true, label="True Brake_f", linewidth=1.2)
    plt.plot(x, brk_pred, label="Pred Brake_f", linestyle="--", linewidth=1.0)
    plt.ylabel("Brake_f")
    plt.legend()
    plt.grid(True)

    # 6) Brake diff
    plt.subplot(6,1,6)
    plt.plot(x, diff_brk, label="Δ Brake_f (pred-true)", linewidth=1.2)
    plt.axhline(0, color="black", linewidth=0.8)
    plt.ylabel("Δ Brake_f")
    plt.xlabel("Time step")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # =================  diff =================
    result = []
    corner_idx = np.array(corner_idx)

    print("\n===== Per-Corner （pred - true）=====")
    for c in sorted(set(corner_idx)):
        if c == 0:
            continue
        mask = (corner_idx == c)
        if mask.sum() < 3:
            continue

        s_mean = diff_steer[mask].mean()
        t_mean = diff_ath[mask].mean()
        b_mean = diff_brk[mask].mean()

        print(
            f"corner {c:2d}: "
            f"steer_diff={s_mean: .2f} deg, "
            f"throttle_diff={t_mean: .2f}, "
            f"brake_diff={b_mean: .2f}"
        )

        result.append({
            "corner": int(c),
            "steer_diff_mean": float(s_mean),
            "throttle_diff_mean": float(t_mean),
            "brake_diff_mean": float(b_mean),
        })

    #  list， DataFrame
    return result

if slow_seqs:
    seq0 = slow_seqs[1]  # 
    corner_table = analyze_one_lap_diff(
        seq0, model, act_mean, act_std,
        title="Slow lap analysis"
    )

    #  per-corner diff 
    corner_df = pd.DataFrame(corner_table)
    display(corner_df)
